In [17]:
import os
import json
import numpy as np
import pandas as pd

import WUBRG
from WUBRG import COLOR_ALIASES_SUPPORT, COLOR_ALIASES, COLOUR_GROUPINGS, COLOUR_GROUPS, COLORS

import set_info
from set_info import SETS, FORMATS, STAT_NAMES
DATA_CACHE = None
PANDAS_CACHE = None
print(f'SETS:          {str(SETS)}')
print(f'FORMATS:       {str(FORMATS)}')


COLUMNS_TRUNC = ["Color", "ALSA", "# GP", "GP WR", "# GIH", "GIH WR"]

SETS:          ['MID']
FORMATS:       ['PremierDraft', 'TradDraft']


In [22]:
set_info.init_cache()
DATA_CACHE = set_info.DATA_CACHE
PANDAS_CACHE = set_info.PANDAS_CACHE

Checking for new format data...
MID PremierDraft data is up to date!
MID TradDraft data is up to date!
Done checking for new format data.

Checking for cache updates...
Parsing MID_PremierDraft.json...
MID PremierDraft added to DATA_CACHE!
Parsing MID_TradDraft.json...
MID TradDraft added to DATA_CACHE!
Done checking for cache updates.



In [30]:
# Filters out cards that have a number of games played less than p% of # GP.
def min_play_filter(df, p):
    min_game = df['# GP'].mean() * (p/100)
    return df[df['# GP'] >= min_game]

# Filters out cards that don't fall within the colour identy.
def color_id_filter(df, color_string):
    if color_string is not None and color_string != '':
        return df[df['Color'].apply(lambda x: set(x) <= set(color_string))]
    else:
        return df

# Filters out cards that aren't in the given rarities.
def rarity_filter(df, rarity):
    if rarity is not None and rarity != '':
        return df[df['Rarity'].apply(lambda x: set(x) <= set(rarity))]
    else:
        return df

# Gets the top n cards, based on a particular stat column.
# Can filter based on card colours, rarity.
# Can get the bottom n cards with 'reverse=True'
def get_top(df, col, n=5, card_colors=None, card_rarity=None, min_thresh=10, reverse=False):
    filtered = min_play_filter(df, min_thresh)
    filtered = color_id_filter(filtered, card_colors)
    filtered = rarity_filter(filtered, card_rarity)

    # Functional XOR.
    reverse = reverse != (col == "ALSA" or col == "ATA")
    
    # Return the smallest values if we're dealing with pick orders
    if reverse != (col == "ALSA" or col == "ATA"):
        return filtered.nsmallest(n, col)
    else:
        return filtered.nlargest(n, col)

# Gets a suite of data about the given colour's position in the metagame.
def get_color_metadata(s, f, color, stat='GIH WR', columns=None):
    df = PANDAS_CACHE[s][f]['']
    if columns == None:
        columns = list(df)
    
    top_commons = get_top(df, stat, n=10, card_colors=color, card_rarity='C')
    top_commons.columns.name = f'Top 10 {color} Commons'
    top_commons = top_commons[columns]
    top_uncommons = get_top(df, stat, n=10, card_colors=color, card_rarity='U')
    top_uncommons.columns.name = f'Top 10 {color} Uncommons'
    top_uncommons = top_uncommons[columns]
    
    color_pairs = [color + x for x in COLORS if x != color]
    color_pair_dfs = dict()
    
    for col in color_pairs:
        c = WUBRG.get_color_identity(col)
        commons = get_top(PANDAS_CACHE[s][f][c], stat, n=5, card_rarity='C')
        uncommons = get_top(PANDAS_CACHE[s][f][c], stat, n=5, card_rarity='U')
        df = pd.concat([commons, uncommons])
        df.columns.name = f'{col} Top Cards'
        df = df[columns]
        color_pair_dfs[c] = df
        
    # TODO: Add in a fun fact here.
        
    return (top_commons, top_uncommons, color_pair_dfs)

In [31]:
s = SETS[0]
f = FORMATS[0]
get_color_metadata(s, f, 'G', columns=COLUMNS_TRUNC)

(Top 10 G Commons      Color   ALSA   # GP   GP WR  # GIH  GIH WR
 Eccentric Farmer          G  6.369  44986  56.142  31193  58.311
 Shadowbeast Sighting      G  4.909  50300  55.410  31187  56.889
 Evolving Wilds               4.820  85467  56.171  47674  56.557
 Silver Bolt                  5.446  63683  55.801  33195  55.641
 Harvesttide Sentry        G  6.321  39863  55.214  24146  55.421
 Duel for Dominance        G  4.795  44763  54.474  25917  54.404
 Might of the Old Ways     G  7.449  21709  54.655  10511  54.191
 Dawnhart Rejuvenator      G  6.278  26691  53.336  14954  54.133
 Jack-o'-Lantern              8.163   8978  53.096   4519  53.906
 Timberland Guide          G  6.614  31198  54.324  17122  53.750,
 Top 10 G Uncommons   Color   ALSA   # GP   GP WR  # GIH  GIH WR
 Rise of the Ants         G  4.065  25067  57.235  12629  61.002
 Deathbonnet Sprout       G  3.941  20554  56.135  10691  59.115
 Dawnhart Mentor          G  3.915  21641  56.499  10559  59.078
 Clear Shot  

In [4]:
PANDAS_CACHE[s][f]['']

TypeError: 'NoneType' object is not subscriptable

In [ ]:
def gt(x, y):  #Used for function pointer shenanigans
    return x > y

def lt(x, y):  #Used for function pointer shenanigans
    return x < y

def filter_qudrant_cards_df(df, rarity, iwd_thresh, func1, func2, cols=None):
    by_rarity = df[df['Rarity'] == rarity]
    games_played_mean = by_rarity['# GP'].mean()
    cards = by_rarity[func1(by_rarity['IWD'], iwd_thresh)]
    cards = cards[func2(cards['# GP'], games_played_mean)]
    cards = cards.sort_values('IWD', ascending=func1==lt)
    if cols is not None:
        cards = cards[cols]
    return cards


def get_trap_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, lt, gt, cols)

def get_trap_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_trap_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_niche_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, gt, lt, cols)

def get_niche_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_niche_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_staple_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, gt, gt, cols)

def get_staple_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_staple_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_dreg_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, lt, lt, cols)

def get_dreg_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_dreg_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)

In [ ]:
def filter_df(df, rarity, color):
    temp = df[df['Rarity'] == rarity]
    temp = temp[temp['Color'] == color]
    return temp

In [ ]:
premier = PANDAS_CACHE['MID']['PremierDraft']['WU'].sort_values('GP WR', ascending=False)
premier

test = premier[premier['Color'].apply(lambda x: set(x) <= set('WU'))]
#frame['Color'].apply(lambda x: any([k in x for k in 'WU']))
min_game = test['# GP'].mean() * 0.1
test[test['# GP'] >= min_game]

In [ ]:
def center(df, stat):
    temp = df.copy()
    
    stat_mean = temp[stat].mean()
    temp[stat] = temp[stat] - stat_mean
    return temp.sort_values(stat, ascending=(stat == "ALSA" or stat == "ATA"))

In [ ]:
center(PANDAS_CACHE['MID']['PremierDraft']['WU'], 'ALSA')

In [ ]:
stat = 'GIH WR'
x = get_top(PANDAS_CACHE['MID']['PremierDraft']['WU'], stat, n=5, card_colors=None, card_rarity='MR')
["Color", "ALSA", "# GP", "GP WR", "# GIH", "GIH WR"]
x

In [ ]:
s = SETS[0]
f = FORMATS[0]
stat = 'GP WR'
#get_top(PANDAS_CACHE[s][f][''], stat, n=5, card_colors=None, card_rarity=None)

rarities = ['', 'C', 'U', 'R', 'M']


def get_top_5s():
    top_5_dict = dict()
    colors = ['W', 'U', 'B', 'R', 'G']
    rarity_sets = [('C', 5), ('U', 5), ('RM', 3)]
    
    for c1 in colors:
        for c2 in colors:
            c = WUBRG.get_color_identity(c1 + c2)
            top_5_dict[c] = dict()
            for r in rarity_sets:
                color_filter = c if (len(c) == 1) else None
                top_5_dict[c][r[0]] = get_top(PANDAS_CACHE[s][f][c], stat, n=r[1], card_colors=color_filter, card_rarity=r[0])
                alias = WUBRG.get_color_alias(c)
                top_5_dict[c][r[0]].columns.name = alias
    return top_5_dict

#top_5_dict = get_top_5s()

def top_5_snippet(color, rarity):
    trgt_cols = ['ALSA', 'ATA', '# GP', 'GP WR', '# GIH', 'GIH WR']
    return top_5_dict[color][rarity][trgt_cols]

#top_5_snippet('UR', 'U')

In [ ]:
premier_commons = premier[premier['Rarity'] == 'C']
premier_commons['GP WR'].nlargest(n=5)

In [ ]:
colours = ''
rarity = 'U'
get_trap_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_niche_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_staple_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_dreg_cards('MID', 'PremierDraft', colours, rarity, -0.005)